<a href="https://colab.research.google.com/github/ertsaz/Taller-Acceso-db/blob/master/03_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Acerca de MySQL
MySQL es un manejador de base de datos de código libre que atyda a los usuarios a almacenar, organizar y recolectar datos. Tiene variedad de opciones para otorgar a determinados usuarios permisos entre tablas y bases de datos, éste tutorial te dará una pequeña introducción a algunas de estas opciones.

In [ ]:
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install pymysql

In [8]:
#@title DATOS DE CONECCIÓN

MYSQL_HOSTNAME = 'test-inv.mysql.database.azure.com' # you probably don't need to change this
USER_CURRENT = '' #@param {type:"string"}
MYSQL_USER = USER_CURRENT+'@test-inv'
MYSQL_PASSWORD = '' #@param {type: "string"}
MYSQL_DATABASE = ''
MYSQL_PORT = 3306
MYSQL_SET = 'mysql+pymysql'

In [ ]:
#@title <- PULSE PARA CONECTAR
%%sql {MYSQL_SET}://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOSTNAME}:{MYSQL_PORT}
SELECT 'BASE DE DATOS CONECTADA'

In [10]:
%reload_ext sql

# 1.Primer Nivel: Mysql
---
> * El primer nivel lo compone el propio Mysql. Debemos de determinar qué usuarios tienen permiso de conexión al servidor Mysql.

> * La información que va a guardar Mysql para determinar si un usuario tiene permiso de conexión es:
* Nombre del usuario
* Password del usuario
* Dirección IP/Nombre del host desde el que el usuario está intentando conectarse al servidor.

Toda esa información está guardada en la tabla user de la base de datos mysql **VEAMOS**.

In [ ]:
%sql SELECT * FROM mysql.user

## 1.1 Creación de usuarios

En las primeras partes del curso de Base de datos, hicimos todas las ediciones como usuario root, con acceso completo a todas las bases de datos. Como sea, en este caso hay mas restricciones que pueden ser requeridas, hay formas de crear usuarios con permisos personalizados.

Vamos empezando por crear un usuario nuevo desde la consola de MySQL:

`CREATE USER 'nombre_usuario'@'localhost' IDENTIFIED BY 'tu_contrasena';`

In [ ]:
%sql CREATE USER 'user'@'%' IDENTIFIED BY '12345678'

Lamentablemente, a este punto el nuevo usuario no tiene permisos para hacer algo con las bases de datos. Por consecuencia si el usuario intenta identificarse (con la contraseña establecida), no será capaz de acceder a la consola de MySQL.

Esto se puede comprobar con el siguiente codigo:

In [ ]:
%sql SHOW CREATE USER 'user'@'%'

### 1.1.1 Opciones para el password
![](https://wiki.cifprodolfoucha.es/images/9/9b/Mysql_usuarios_14.jpg)

* Podemos 'obligar' a que un usuario cambie su contraseña 

    > cuando se conecta por primera vez:
    > 
    > `CREATE USER 'angel'@'localhost' IDENTIFIED BY '12345678'  PASSWORD EXPIRE;`
    > 
    > Poniendo la palabra 'PASSWORD EXPIRE'.



* También podemos 'obligar' al usuario a que cambie de > password cada cierto número de días:
> 
> `CREATE USER 'angel'@'localhost' IDENTIFIED BY '12345678' PASSWORD EXPIRE INTERVAL 60 DAY;`
> 
> En el ejemplo, deberá cambiar de password cada 60 días.


In [ ]:
%sql CREATE USER 'user'@'%' IDENTIFIED BY '12345678' PASSWORD EXPIRE

### 1.1.2 Bloqueo / Desbloqueo de cuentas

![](https://wiki.cifprodolfoucha.es/images/6/6a/Mysql_usuarios_15.jpg)

Cuando creamos (o modificamos) una cuenta, podemos indicar si dicha cuenta está bloqueada.
En caso de estarlo, no estaría permitida la conexión al servidor Mysql haciendo uso de dicha cuenta.

Por ejemplo:

In [ ]:
%sql CREATE USER 'user'@'%' IDENTIFIED BY '12345678' ACCOUNT LOCK;

**Probemos  y conectemonos!!**

In [ ]:
#@title DATOS DE CONECCIÓN

MYSQL_HOSTNAME = 'test-inv.mysql.database.azure.com' # you probably don't need to change this
USER_CURRENT = '' #@param {type:"string"}
MYSQL_USER = USER_CURRENT+'@test-inv'
MYSQL_PASSWORD = '' #@param {type: "string"}
MYSQL_DATABASE = ''
MYSQL_PORT = 3306
MYSQL_SET = 'mysql+pymysql'

In [ ]:
#@title <- PULSE PARA CONECTAR
%%sql {MYSQL_SET}://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOSTNAME}:{MYSQL_PORT}
SELECT 'BASE DE DATOS CONECTADA'

### 1.1.3 Opciones de limitación de recursos

Cuando creamos una cuenta podemos establecer ciertos límites al uso que un usuario pueda hacer de los recursos del servidor Mysql.
Un ejemplo:

In [ ]:
%sql DROP USER 'user'@'%' -- reseteando el usuario
%sql CREATE USER 'user'@'%' IDENTIFIED BY '12345678' WITH MAX_QUERIES_PER_HOUR 2 MAX_USER_CONNECTIONS 2;

In [ ]:
#@title DATOS DE CONECCIÓN

MYSQL_HOSTNAME = 'test-inv.mysql.database.azure.com' # you probably don't need to change this
USER_CURRENT = '' #@param {type:"string"}
MYSQL_USER = USER_CURRENT+'@test-inv'
MYSQL_PASSWORD = '' #@param {type: "string"}
MYSQL_DATABASE = ''
MYSQL_PORT = 3306
MYSQL_SET = 'mysql+pymysql'

In [ ]:
#@title <- PULSE PARA CONECTAR
%%sql {MYSQL_SET}://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOSTNAME}:{MYSQL_PORT}
SELECT 'BASE DE DATOS CONECTADA'

## 1.2 Modificando usuarios

* Como vemos la sintaxis es casi idéntica a la del CREATE 
USER, con las mismas opciones, lo que pasa es que estaríamos modificando los valores de dichas opciones para un usuario existente.

* Lo único diferente es la opción IF EXISTS. Dicha opción sirve para que el gestor Mysql realice la orden SQL 'ALTER USER' sólo en el caso de que el usuario exista. Si no existe no hace nada.

  > Si no ponemos esta opción a la hora de modificar un   usuario, tendríamos un mensaje de error.
  Si estamos ejecutando un conjunto de instrucciones Mysql,   pararía al llegar a esta instrucción.

Un ejemplo: Para modificar el nombre del usuario debemos hacer uso de la orden SQL: RENAME USER

In [ ]:
%sql RENAME USER 'user'@'%' to 'user_act'@'%'

## 1.3 Borrando usuarios

![](https://wiki.cifprodolfoucha.es/images/4/42/Mysql_usuarios_23.jpg)

La orden SQL que permite dar de baja a usuarios es: DROP USER

Un ejemplo:

In [ ]:
%sql DROP USER IF EXISTS'user'@'%' 

# 2.Segundo Nivel: Bases de datos
---
> * Una vez creado el usuario debemos otorgarle permisos.
Los más comunes son los que le van a permitir realizar operaciones sobre una base de datos.

> * Cuando creamos un usuario podemos otorgarle permisos a nivel de 'todas las bases de datos' o a nivel de 'una base de datos en particular'.

***Los permisos globales*** (los que se aplican a todas las bases de datos) están definidos en la propia tabla 'user' de la base de datos 'mysql'

In [ ]:
%sql DESC mysql.user

***Los permisos locales*** a nivel de una base de datos en particular se encuentran guardados en la tabla db de la base de datos mysql.

In [ ]:
%sql DESC mysql.db

Las columnas Host,Db, User conforman la clave primaria de la tabla y por tanto no puede haber dos filas con los mismos valores.
* Host: nombre del host desde está permitido acceder a la base de datos.
* Db: nombre de la base de datos.
* User: nombre del usuario que tiene permiso de acceso a la base de datos.



In [ ]:
%sql SELECT Host, User, Db, Select_priv FROM mysql.db

## 2.1 GRANT Permisos a nivel global

> Para que otorgar un permiso a este nivel, hay que indicar: ON `*.*` en la cláusula grant.

La sintaxis básica para dar permisos a nivel del gestor Mysql, incluyendo los permisos administrativos sería:

In [ ]:
%sql GRANT SELECT ON *.* TO '___'@'___'

In [ ]:
%sql SELECT * FROM mysql.user -- ejecutamos para ver los cambios

Podemos otorgar varios permisos en una única operación, separándolos con comas:

In [ ]:
%sql GRANT SELECT,UPDATE ON *.* TO '___'@'___'

In [ ]:
%sql SELECT * FROM mysql.user -- ejecutamos para ver los cambios

## 2.2 Permisos a nivel de bases de datos

> Recordar que a este nivel, los permisos son filas que se añaden, borran o se modifcan en la tabla mysql.db, y se aplican en una base de datos concreta.

In [ ]:
GRANT ALL ON mydb.mytbl TO '___'@'___'

# 3.Tercer Nivel: Tablas y columnas
---

## 3.1 Permisos a nivel de tabla
> Recordar que a este nivel, los permisos son filas que se añaden, borran o se modifican en la tabla mysql.tables_priv, y se aplican en una base de datos concreta y en una tabla concreta.

In [ ]:
GRANT ALL ON mydb.mytbl TO 'someuser'@'somehost';

## 3.2 Permisos a nivel de columnas
> Recordar que a este nivel, los permisos son filas que se añaden, borran o se modifican en la tabla mysql.tables_priv, y se aplican en una base de datos concreta y en una tabla concreta.

`GRANT SELECT (col1), INSERT (col1,col2) ON mydb.mytbl TO 'someuser'@'somehost';`